# 1. Prerequisite

1. Importing required packages
2. Reading CSV and creating Data Frame
3. Understand the Data Frame

In [36]:
#Importing Required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 50)

In [37]:
# Creating dataframe from CSV file
loan = pd.read_csv("./loan/loan.csv")

In [ ]:
# Checking the data
loan.head()

In [ ]:
# Understanding the shape of the DF
loan.shape

In [ ]:
# Get the details of the data
loan.info()

# 2. Data Cleaning

1. Drop the columns if,
    1. The missing values for that column is > 40%
    2. The column having only one value (unique count is 1) 
    3. The percentage of unique values in a column is less than 2% (e.g. URL - all values are unique)
2. Drop the rows,
    1. We are considering only Closed and Chargged of loans. So remving the loan with status 'Current'.

In [ ]:
# Find isnull values in dataset and get the percentage 
missing_values_percentage = loan.isnull().sum() / len(loan) * 100
print(missing_values_percentage)

In [ ]:
# Get coloumns with more than 40% of data
columns_with_missing_data = missing_values_percentage[missing_values_percentage > 40].index
print(columns_with_missing_data)

In [43]:
# Drop the columns with more than 40% of miss_dataing data
loan_data = loan.drop(columns=columns_with_missing_data)

In [ ]:
loan_data.shape

In [ ]:
# Checking for the coloumns that still have some missing values
missing_value = loan_data.isnull().sum() / len(loan_data) * 100
find_missing_columns = missing_value[missing_value > 0]
print(find_missing_columns)

In [ ]:
# As we have reduce to 54 columns after data cleaning.
# Now we will again check for the columns with the unique counts in them.
unique_col = loan_data.nunique()
print(unique_col)

In [47]:
# As per the above observation we got the columns with single unique value,
# so it will not affect the analysis
# remove the columns with unique count = 1
uni_col = unique_col[unique_col.values == 1]
unique_col_list = uni_col.index.tolist()
loan_data.drop(columns=unique_col_list, inplace=True)

In [ ]:
loan_data.shape

In [ ]:
# Columns where the number of unique values is greater than 2% 
per_unique_value = loan_data.nunique()
per_uni_col = per_unique_value[per_unique_value > len(loan_data)* .02]
print(per_uni_col)

In [50]:
cols_not_required = ["url", "desc", "title", "emp_title", "zip_code", "out_prncp", "out_prncp_inv", "id", "member_id"]
loan_data.drop(columns=cols_not_required, inplace=True)

In [ ]:
loan_data.shape

### 2.1. Data Preparation 

In [52]:
#Remove rows where the loan_status is current as current means in-porgress
rows_to_drop = loan_data[loan_data['loan_status'] == "Current"].index
loan_data.drop(rows_to_drop, inplace=True)

In [ ]:
loan_data.shape
# loan_data.head()
# loan_data.info()

# 3. Univariate Analysis